In [321]:
import json
import sys
import dropbox
import requests
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib
import pyvis

from pyvis.network import Network

In [322]:
ws_pathlist = json.loads(
    requests.get(
        "https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/whoscored-matchlist.json"
    ).content
)

In [323]:
dic = json.loads(requests.get(ws_pathlist[2]).content)

In [324]:
dic.keys()

dict_keys(['playerIdNameDictionary', 'periodMinuteLimits', 'timeStamp', 'attendance', 'venueName', 'referee', 'weatherCode', 'elapsed', 'startTime', 'startDate', 'score', 'htScore', 'ftScore', 'etScore', 'pkScore', 'statusCode', 'periodCode', 'home', 'away', 'maxMinute', 'minuteExpanded', 'maxPeriod', 'expandedMinutes', 'expandedMaxMinute', 'periodEndMinutes', 'commonEvents', 'events', 'timeoutInSeconds'])

In [325]:
def getLastItem(dictionary):
    last_keyval = dictionary.popitem()
    dictionary.update({last_keyval[0]:last_keyval[1]})
    return {last_keyval[0]:last_keyval[1]}

# Adatmaszírozás

In [326]:
team_id_dic = {
   dic[v]["teamId"]: f"{v} - {dic[v]['name']}" for v in ["home", "away"]
}

In [327]:
passes = []
for i in range(len(dic["events"])):
    if (
        dic["events"][i]["type"]["displayName"] == "Pass"
        and dic["events"][i]["outcomeType"]["displayName"] == "Successful"
        and dic["events"][i]["teamId"] == dic["events"][i + 1]["teamId"]
    ):
        dic["events"][i]["recieverId"] = dic["events"][i + 1]["playerId"]
        passes.append(dic["events"][i])

passes_df_big = pd.DataFrame(passes)

passes_df = passes_df_big[["playerId", "recieverId", "teamId"]]

passes_df["passes"] = "huh"

test_df = passes_df.groupby(["playerId", "recieverId", "teamId"]).count().reset_index()

test_df = test_df[test_df["playerId"] != test_df["recieverId"]].reset_index(drop=True)

test_df["color"] = test_df.apply(
    lambda x: "red" if x["teamId"] == dic["home"]["teamId"] else "blue", axis=1
)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [328]:
sub_1=passes_df_big[["playerId", "recieverId", "x", "y", 'endX', "endY"]]

sub_2=pd.DataFrame()

sub_2['playerId']=sub_1['recieverId']
sub_2['recieverId']=sub_1['playerId']
sub_2['x']=sub_1['endX']
sub_2['y']=sub_1['endY']
sub_2['endX']=sub_1['x']
sub_2['endY']=sub_1['y']

sub=pd.concat([sub_1,sub_2])

sub=sub.drop_duplicates(subset=['playerId'])

coordinates=sub[['playerId', 'x', 'y']]

In [329]:
dic['home']['players'][1]["stats"]["ratings"]

{'0': 5.99,
 '2': 6.0,
 '4': 6.03,
 '5': 6.03,
 '6': 6.03,
 '9': 6.03,
 '12': 6.01,
 '14': 6.1,
 '16': 6.09,
 '18': 6.1,
 '20': 6.09,
 '21': 6.14,
 '26': 6.19,
 '28': 6.2,
 '34': 6.03,
 '36': 6.04,
 '43': 6.03,
 '47': 6.02,
 '51': 6.03,
 '53': 6.03,
 '54': 5.86,
 '55': 5.85,
 '56': 5.85,
 '58': 5.85,
 '59': 5.83,
 '60': 5.84,
 '61': 5.86,
 '64': 5.84,
 '66': 5.83,
 '67': 5.82,
 '68': 5.83,
 '69': 5.81,
 '71': 5.82,
 '76': 5.77,
 '79': 5.76,
 '80': 5.85,
 '81': 6.01,
 '82': 6.0,
 '84': 6.09,
 '86': 6.07,
 '87': 6.24,
 '88': 6.28,
 '89': 6.28,
 '90': 6.29,
 '91': 6.4,
 '92': 6.4,
 '93': 6.41,
 '94': 6.42,
 '97': 6.42,
 '99': 6.42,
 '100': 6.68}

In [330]:
test_df['playerId']

0        9446
1        9446
2        9446
3        9446
4        9446
        ...  
186    363966
187    363966
188    363966
189    384566
190    384566
Name: playerId, Length: 191, dtype: int64

In [331]:
for i in dic['home']['players']:
    if i['playerId'] in list(test_df['playerId']) or i['playerId'] in list(test_df['recieverId']): 
        print(i['stats']['ratings'].popitem()[1])

6.07
6.68
7.55
6.32
6.01
6.47
6.64
6.94
10.0
6.78
7.76
6.26
6.28
6.13


In [332]:
players = []
for i in dic["home"]["players"]:
        if i['playerId'] in list(test_df['playerId']) or i['playerId'] in list(test_df['recieverId']): 
            players.append([i["playerId"], i["name"], i['shirtNo'], i['position'], i['height'], i['weight'],
                            i['age'], i['stats']['ratings'].popitem()[1]])
players = pd.DataFrame.from_records(players).rename(columns={0: "playerId", 1: "name", 2:"shirtNo", 3:"position",
                                                            4:'height', 5: 'weight', 6:"age", 7:"rating"})
home_players = players.copy()

players = []
for i in dic["away"]["players"]:
        if i['playerId'] in list(test_df['playerId']) or i['playerId'] in list(test_df['recieverId']): 
            players.append([i["playerId"], i["name"], i['shirtNo'], i['position'], i['height'], i['weight'],
                            i['age'], i['stats']['ratings'].popitem()[1]])
players = pd.DataFrame.from_records(players).rename(columns={0: "playerId", 1: "name", 2:"shirtNo", 3:"position",
                                                            4:'height', 5: 'weight', 6:"age", 7:"rating"})
away_players = players

players = pd.concat([home_players, away_players]).reset_index().drop("index", axis=1)

In [333]:
test_df["playerName"] = "huh"
test_df["recieverName"]= "huh"
for i in range(len(test_df["playerId"])):
    test_df["playerName"][i] = (
        players["name"]
        .loc[players["playerId"] == test_df["playerId"][i]]
        .reset_index(drop=True)[0]
    )
    test_df["recieverName"][i] = (
        players["name"]
        .loc[players["playerId"] == test_df["recieverId"][i]]
        .reset_index(drop=True)[0]
    )

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [334]:
test_df=test_df.merge(coordinates)

In [335]:
test_df

,playerId,recieverId,teamId,passes,color,playerName,recieverName,x,y
0,9446,31958,67,4,red,Jesús Navas,Fernando,53.2,24.8
1,9446,33724,67,7,red,Jesús Navas,Éver Banega,53.2,24.8
2,9446,68585,67,2,red,Jesús Navas,Luuk de Jong,53.2,24.8
3,9446,68732,67,1,red,Jesús Navas,Tomás Vaclik,53.2,24.8
4,9446,77417,67,1,red,Jesús Navas,Nemanja Gudelj,53.2,24.8
...,...,...,...,...,...,...,...,...,...
186,363966,133445,70,4,blue,Adrià Pedrosa,Jonathan Calleri,30.3,100.0
187,363966,135572,70,4,blue,Adrià Pedrosa,Adrián Embarba,30.3,100.0
188,363966,323148,70,4,blue,Adrià Pedrosa,Marc Roca,30.3,100.0
189,384566,106596,70,1,blue,Víctor Gómez,Sergi Darder,78.1,27.4


In [336]:
players

,playerId,name,shirtNo,position,height,weight,age,rating
0,68732,Tomás Vaclik,1,GK,188,85,31,5.97
1,9446,Jesús Navas,16,DR,172,60,34,6.42
2,134984,Diego Carlos,20,DC,185,79,27,7.54
3,90778,Sergi Gómez,3,DC,185,80,28,6.31
4,362275,Sergio Reguilón,23,DL,178,68,23,6.02
5,33724,Éver Banega,10,MC,174,71,31,6.46
6,102439,Franco Vázquez,22,MC,187,82,31,6.47
7,31958,Fernando,25,MC,183,76,32,6.94
8,105591,Suso,14,FWR,177,70,26,9.85
9,323547,Youssef En-Nesyri,15,FW,189,73,22,6.88


In [337]:
str(players.iloc[0])

'playerId           68732\nname        Tomás Vaclik\nshirtNo                1\nposition              GK\nheight               188\nweight                85\nage                   31\nrating              5.97\nName: 0, dtype: object'

In [338]:
G = nx.DiGraph()
for i in range(1, (len(test_df))):
    G.add_edge(
        test_df.loc[i, "playerName"],
        test_df.loc[i, "recieverName"],
        weight=test_df.loc[i, "passes"],
    )

cc = nx.closeness_centrality(G, u=None, distance=None, wf_improved=False)

CC_df = pd.DataFrame.from_dict(cc, orient="index", columns=["CC"])

test_df = test_df.merge(CC_df, how="left", left_on="playerName", right_index=True)

In [339]:
test_df.to_pickle("test_df.pkl")

In [340]:
test_df

,playerId,recieverId,teamId,passes,color,playerName,recieverName,x,y,CC
0,9446,31958,67,4,red,Jesús Navas,Fernando,53.2,24.8,0.764706
1,9446,33724,67,7,red,Jesús Navas,Éver Banega,53.2,24.8,0.764706
2,9446,68585,67,2,red,Jesús Navas,Luuk de Jong,53.2,24.8,0.764706
3,9446,68732,67,1,red,Jesús Navas,Tomás Vaclik,53.2,24.8,0.764706
4,9446,77417,67,1,red,Jesús Navas,Nemanja Gudelj,53.2,24.8,0.764706
...,...,...,...,...,...,...,...,...,...,...
186,363966,133445,70,4,blue,Adrià Pedrosa,Jonathan Calleri,30.3,100.0,0.666667
187,363966,135572,70,4,blue,Adrià Pedrosa,Adrián Embarba,30.3,100.0,0.666667
188,363966,323148,70,4,blue,Adrià Pedrosa,Marc Roca,30.3,100.0,0.666667
189,384566,106596,70,1,blue,Víctor Gómez,Sergi Darder,78.1,27.4,0.631579


# háló

In [341]:
test_df = pd.read_pickle('test_df.pkl')

In [342]:
sevilla_logo  = "https://www.logofootball.net/wp-content/uploads/Sevilla-FC-Logo.png"
espanyol_logo = "https://upload.wikimedia.org/wikipedia/en/thumb/d/d6/Rcd_espanyol_logo.svg/1200px-Rcd_espanyol_logo.svg.png"

In [343]:
test_df['logo']=test_df.apply(lambda x: espanyol_logo if x['teamId']==70 else sevilla_logo, axis=1 )

In [344]:
players.columns

Index(['playerId', 'name', 'shirtNo', 'position', 'height', 'weight', 'age',
       'rating'],
      dtype='object')

In [345]:
players.loc[2,'playerId']

134984

In [346]:
nev_lista=list(players['name'])
nev_lista

['Tomás Vaclik',
 'Jesús Navas',
 'Diego Carlos',
 'Sergi Gómez',
 'Sergio Reguilón',
 'Éver Banega',
 'Franco Vázquez',
 'Fernando',
 'Suso',
 'Youssef En-Nesyri',
 'Lucas Ocampos',
 'Luuk de Jong',
 'Nemanja Gudelj',
 'Nolito',
 'Diego López',
 'Javi López',
 'Bernardo Espinosa',
 'Leandro Cabrera',
 'Adrià Pedrosa',
 'Víctor Sánchez',
 'Marc Roca',
 'Wu Lei',
 'Sergi Darder',
 'Adrián Embarba',
 'Jonathan Calleri',
 'Facundo Ferreyra',
 'Fernando Calero',
 'Víctor Gómez']

In [347]:
faszom=players.transpose()
faszom=faszom.set_axis(nev_lista, axis=1, inplace=False)

In [348]:
faszom

,Tomás Vaclik,Jesús Navas,Diego Carlos,Sergi Gómez,Sergio Reguilón,Éver Banega,Franco Vázquez,Fernando,Suso,Youssef En-Nesyri,...,Adrià Pedrosa,Víctor Sánchez,Marc Roca,Wu Lei,Sergi Darder,Adrián Embarba,Jonathan Calleri,Facundo Ferreyra,Fernando Calero,Víctor Gómez
playerId,68732,9446,134984,90778,362275,33724,102439,31958,105591,323547,...,363966,33556,323148,127220,106596,135572,133445,125293,362354,384566
name,Tomás Vaclik,Jesús Navas,Diego Carlos,Sergi Gómez,Sergio Reguilón,Éver Banega,Franco Vázquez,Fernando,Suso,Youssef En-Nesyri,...,Adrià Pedrosa,Víctor Sánchez,Marc Roca,Wu Lei,Sergi Darder,Adrián Embarba,Jonathan Calleri,Facundo Ferreyra,Fernando Calero,Víctor Gómez
shirtNo,1,16,20,3,23,10,22,25,14,15,...,3,4,21,7,10,23,12,9,24,34
position,GK,DR,DC,DC,DL,MC,MC,MC,FWR,FW,...,DL,DMC,DMC,AMR,AMC,AML,FW,Sub,Sub,Sub
height,188,172,185,185,178,174,187,183,177,189,...,176,174,184,174,180,173,181,183,183,166
weight,85,60,79,80,68,71,82,76,70,73,...,69,72,74,66,71,66,72,79,71,69
age,31,34,27,28,23,31,31,32,26,22,...,21,32,23,28,26,27,26,29,24,20
rating,5.97,6.42,7.54,6.31,6.02,6.46,6.47,6.94,9.85,6.88,...,6.62,5.14,5.95,7.72,6.62,7.52,7.88,6.39,5.97,5.97


In [349]:
faszom=faszom.set_axis(list("<br>"+faszom.index), axis=0, inplace=False)

In [350]:
faszom

,Tomás Vaclik,Jesús Navas,Diego Carlos,Sergi Gómez,Sergio Reguilón,Éver Banega,Franco Vázquez,Fernando,Suso,Youssef En-Nesyri,...,Adrià Pedrosa,Víctor Sánchez,Marc Roca,Wu Lei,Sergi Darder,Adrián Embarba,Jonathan Calleri,Facundo Ferreyra,Fernando Calero,Víctor Gómez
<br>playerId,68732,9446,134984,90778,362275,33724,102439,31958,105591,323547,...,363966,33556,323148,127220,106596,135572,133445,125293,362354,384566
<br>name,Tomás Vaclik,Jesús Navas,Diego Carlos,Sergi Gómez,Sergio Reguilón,Éver Banega,Franco Vázquez,Fernando,Suso,Youssef En-Nesyri,...,Adrià Pedrosa,Víctor Sánchez,Marc Roca,Wu Lei,Sergi Darder,Adrián Embarba,Jonathan Calleri,Facundo Ferreyra,Fernando Calero,Víctor Gómez
<br>shirtNo,1,16,20,3,23,10,22,25,14,15,...,3,4,21,7,10,23,12,9,24,34
<br>position,GK,DR,DC,DC,DL,MC,MC,MC,FWR,FW,...,DL,DMC,DMC,AMR,AMC,AML,FW,Sub,Sub,Sub
<br>height,188,172,185,185,178,174,187,183,177,189,...,176,174,184,174,180,173,181,183,183,166
<br>weight,85,60,79,80,68,71,82,76,70,73,...,69,72,74,66,71,66,72,79,71,69
<br>age,31,34,27,28,23,31,31,32,26,22,...,21,32,23,28,26,27,26,29,24,20
<br>rating,5.97,6.42,7.54,6.31,6.02,6.46,6.47,6.94,9.85,6.88,...,6.62,5.14,5.95,7.72,6.62,7.52,7.88,6.39,5.97,5.97


In [352]:
def pass_net(df,player_df,height="800px",width="100%",name="team"):
    pass_net = Network(
        height=height,
        width=width,
        bgcolor="FFFFFF",
        font_color="black",
        directed=True,
        notebook=False,
    )
    
    pass_net.barnes_hut()
    sources = df["playerName"]
    targets = df["recieverName"]
    weights = df["passes"]
    size = df["CC"]
    logo = df["logo"]
    x=df["x"]
    y=df["y"]
    #shape='image', image =
    edge_data = zip(sources, targets, weights, size, logo, x, y)
    for e in edge_data:
        src = str(e[0])
        dst = str(e[1])
        w = e[2]
        s = e[3]
        i = e[4]
        x = e[5]
        y = e[6]
        pass_net.add_node(src, src, title=src, size=s, value=s*3,shape='image', image=i ,x=x , y=y)
        pass_net.add_node(dst, dst, title=dst, size=s, value=s*3,shape='image', image=i ,x=x , y=y)
        pass_net.add_edge(src, dst, value=w)
        neighbor_map = pass_net.get_adj_list()
    
    for node in pass_net.nodes:
        node["title"] +=str(player_df[node['title']])
        
        
    
    pass_net.set_options(
        """
    var options = {
      "nodes": {
        "borderWidth": 2,
        "color": {
          "highlight": {
            "background": "rgba(217,255,50,1)"
          }
        },
        "font": {
          "size": 50,
          "face": "tahoma"
        }
      },
      "edges": {
        "color": {
          "inherit": true
        },
        "smooth": false
      },
      "physics": {
        "barnesHut": {
          "gravitationalConstant": -80000,
          "springLength": 250,
          "springConstant": 0.001
        },
        "minVelocity": 0.75
      }
    }
    """
    )
    
    pass_net.show("pass_network"+name+".html")
    return pass_net

In [34]:
list(team_id_dic.keys())[0]

67

# Dash

In [ ]:
import dash_dangerously_set_inner_html
import dash
import dash_html_components as html
from dash_core_components import Dropdown, Graph
from dash_html_components import Div, H3, H1, Link

external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.title = "Pass map"

app.scripts.config.serve_locally = True

app.layout = html.Div(
    children=[
        H1(
            children="Sevilla – Espanyol pass network",
            style={
                "color": "black",
                "backgroundColor": "ffffff",
                "text-align": "center",
            },
        ),
        html.Div(
            children=[
                html.H3(children="Sevilla", className="six columns"),
                html.Div(
                    html.Iframe(
                        srcDoc=open("pass_networkteam1.html").read(),
                        style={"height": "100%", "width": "100%"},
                    ),
                    style={"height": "400px"},
                    className="six columns",
                ),
            ],
            className="row",
        ),
        html.Div(
            children=[
                html.H3(children="Espanyol", className="six columns"),
                html.Div(
                    html.Iframe(
                        srcDoc=open("pass_networkteam2.html").read(),
                        style={"height": "100%", "width": "100%"},
                    ),
                    style={"height": "400px"},
                    className="six columns",
                ),
            ],
            className="row",
        ),
    ]
)


if __name__ == "__main__":
    app.run_server(debug=True, use_reloader=False, port=8054)

Running on http://127.0.0.1:8054/
Running on http://127.0.0.1:8054/
Running on http://127.0.0.1:8054/
Running on http://127.0.0.1:8054/
Debugger PIN: 302-655-793
Debugger PIN: 302-655-793
Debugger PIN: 302-655-793
Debugger PIN: 302-655-793
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
